In [1]:
from ultralytics import YOLO
import cv2
import time

In [2]:
# Carregar o modelo YOLOv8 pré-treinado
model = YOLO("yolov8n.pt")  # Use "yolov8l.pt" para mais precisão

# Captura de vídeo da webcam
cap = cv2.VideoCapture(0)

# ROI (Region of Interest) para detecção de objetos
roi_start = (100, 100)  # Coordenadas do canto superior esquerdo da área
roi_end = (300, 500)    # Coordenadas do canto inferior direito da área

# Inicializar gravação de vídeo
video_writer = None
recording = False

# Definir limite de confiança
confidence_threshold = 0.80

# Variáveis para monitorar o tempo dentro do ROI
inside_roi = False
start_time = None
elapsed_time = 0

In [3]:
while True:
    # Ler frame da webcam
    ret, frame = cap.read()
    if not ret:
        break
    
    results = model(frame)
    
    # Filtrar detecções para encontrar apenas pessoas com confiança >= 0.80
    person_detected = False
    for result in results:
        for box in result.boxes:
            if box.cls == 0:  # Classe 0 é 'pessoa' no COCO dataset
                confidence = box.conf.item()
                
                if confidence >= confidence_threshold:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                    
                    # Verificar se a pessoa está dentro da área de interesse
                    if (roi_start[0] < x1 < roi_end[0]) and (roi_start[1] < y1 < roi_end[1]):
                        person_detected = True

                        # Iniciar o temporizador se a pessoa entrou no ROI
                        if not inside_roi:
                            start_time = time.time()
                            inside_roi = True
                        
                        # Iniciar gravação se não estiver gravando
                        if not recording:
                            fourcc = cv2.VideoWriter_fourcc(*'XVID')
                            video_writer = cv2.VideoWriter('detected.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))
                            recording = True

                        # Mostrar o tempo de permanência no frame junto com a pessoa
                        elapsed_time = time.time() - start_time
                        time_text = f"Tempo no ROI: {elapsed_time:.2f}s"
                        cv2.putText(frame, time_text, (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

                    # Desenhar o retângulo ao redor da pessoa detectada
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 1)
                    cv2.putText(frame, f"Pessoa {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    
    # Se a pessoa sair do ROI, parar o temporizador
    if inside_roi and not person_detected:
        inside_roi = False
    
    # Desenhar a área de interesse no frame
    cv2.rectangle(frame, roi_start, roi_end, (0, 255, 0), 2)
    
    # Gravar o frame se estiver gravando
    if recording:
        video_writer.write(frame)

    # Mostrar frame
    cv2.imshow("Deteccao de Pessoas", frame)

    # Parar se a tecla 'q' for pressionada
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar a captura e fechar janelas
cap.release()
if recording:
    video_writer.release()
cv2.destroyAllWindows()


0: 480x640 1 person, 1 bench, 1 toilet, 165.7ms
Speed: 4.4ms preprocess, 165.7ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 toilet, 1 tv, 168.7ms
Speed: 0.0ms preprocess, 168.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 160.2ms
Speed: 4.2ms preprocess, 160.2ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 1 tv, 145.0ms
Speed: 2.0ms preprocess, 145.0ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 148.5ms
Speed: 1.7ms preprocess, 148.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 147.8ms
Speed: 3.5ms preprocess, 147.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 1 tv, 148.7ms
Speed: 0.0ms preprocess, 148.7ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 cha